# Question: Is the company a good steward of the environment? 

These days, a company's environmental stewardship is about much more than not actively polluting. Climate change looms large over the globe, and companies that work hard to lighten their carbon footprint are the ones worth considering in our ESG investing framework.

Look for specific disclosures of initiatives and goals, and transparency about whether the company is meeting greenhouse-gas emissions goals. A high grade from nonprofit ratings organization CDP (formerly known as the Carbon Disclosure Project) tells you the company is doing its part for the environment. Companies that pursue the use of (or a complete transition to) renewable energy also get high marks here.

The environmental piece of the puzzle includes responsible sourcing, and dealing with resource scarcity. For example, is a water-intensive company allocating funds and employees to conserve water? Does a consumer-goods company innovate to use fewer materials, or recycled materials?

Internal work is essential, but so is a company's environmental approach to stakeholders, such as customers. Businesses can do good by empowering people to recycle and reuse electronics, or by offering products that help consumers increase their own energy efficiency.

While these initiatives can be costly up front, ultimately these efforts will reduce costs for companies implementing them. Robust sustainability reports usually include how much money a company saved via its sustainability initiatives; such savings fly in the face of conventional wisdom, which says being environmentally responsible is too expensive for companies to pursue. Lastly, while eco-friendly initiatives can generate growth, they can also be used as risk-mitigation tools to prevent lawsuits, fines, and burdensome regulation.

If a company lacks solid environmental initiatives and goals, is a polluter or waster of natural resources, or exhibits other sustainability problems, we might exclude it from consideration for our ESG portfolio.

Source:  
https://www.fool.com/investing/2019/04/09/going-for-great-returns-and-the-greater-good-fools.aspx

# Data Sources

**Source**  
News

**Methodology**  
Because companies are only pushing positive environmental news, I will only search for negative environmental news. I will do this by only using news articles that have a negative polarity score 

# Imports

In [31]:
# All necessary imports
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from textblob import TextBlob

# Companies

In [2]:
# Getting site
page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BeautifulSoup(page.content, 'html.parser')

# Finding table
table = soup.find(id='constituents').tbody

# Getting table rows
table_rows = table.find_all('tr')

# Finding company in row
companies = []
for row in table_rows[1:70]:
    elements = row.find_all('td')
    company = elements[1].text
    companies.append(company)

# Cleaning company rows
cleaned_companies = []
for company in companies: 
    if company == 'Alphabet Inc. (Class A)':
        pass
    elif '(Class B)' in company:
        company = company.replace('(Class B)','')
        cleaned_companies.append(company)
    elif '(Class C)' in company:
        company = company.replace('(Class C)','')
        cleaned_companies.append(company)
    else:
        cleaned_companies.append(company)

# Defining final companies
companies = cleaned_companies

In [4]:
# Checking companies
companies[:10]

['3M Company',
 'Abbott Laboratories',
 'AbbVie Inc.',
 'Abiomed',
 'Accenture',
 'Activision Blizzard',
 'Adobe Inc.',
 'Advanced Micro Devices',
 'Advance Auto Parts',
 'AES Corp']

In [37]:
# Set for script automation
inputs = companies

# Getting Negative News Data

In [33]:
def getting_neg_env_data(companies):

    # Reading in api key
    api_key = open('/Users/MichaelWirtz/Desktop/pathfile/newsapi_key.txt').read()
    # Inputing api key
    newsapi = NewsApiClient(api_key=api_key)

    polarity_df = pd.DataFrame(columns=['company','neg_env_count'])

    for company in companies:
        # Get news articles
        envi_company = newsapi.get_everything(
                       q='environment {}'.format(company),
                       language='en',
                       sort_by='relevancy')

        # Empty list for urls
        url_list = []
        # Appending urls to list
        for i in range(0, len(envi_company['articles'])):
            url = env_disaster['articles'][i]['url']
            url_list.append(url)

        # Getting polarity ratings of articles
        negative_polarity_count = 0
        for url in url_list:
            res = requests.get(url)
            html_page = res.content
            soup = BeautifulSoup(html_page, 'html.parser')
            text = soup.find_all(text=True)

            output = ''
            blacklist = [
                '[document]',
                'noscript',
                'header',
                'html',
                'meta',
                'head', 
                'input',
                'script',
            ]

            for t in text:
                if t.parent.name not in blacklist:
                    output += '{} '.format(t)

            output = re.sub(r"[^A-Za-z]+",' ',output)
            blob = TextBlob(output)
            polarity = blob.polarity

            if polarity < 0:
                negative_polarity_count += 1

        polarity_df = polarity_df.append({'company': company,
                                          'neg_env_count': negative_polarity_count}, ignore_index=True)

        print(company)
    
    return polarity_df

In [34]:
env_data = getting_neg_env_data(companies)

3M Company
Abbott Laboratories
AbbVie Inc.
Abiomed
Accenture
Activision Blizzard
Adobe Inc.
Advanced Micro Devices
Advance Auto Parts
AES Corp
Aflac
Agilent Technologies
Air Products & Chemicals
Akamai Technologies
Alaska Air Group
Albemarle Corporation
Alexandria Real Estate Equities
Alexion Pharmaceuticals
Align Technology
Allegion
Alliant Energy
Allstate Corp
Alphabet Inc. 
Altria Group Inc
Amazon.com Inc.
Amcor
Ameren Corp
American Airlines Group
American Electric Power
American Express
American International Group
American Tower Corp.
American Water Works
Ameriprise Financial
AmerisourceBergen
Ametek
Amgen Inc.
Amphenol Corp
Analog Devices, Inc.
ANSYS, Inc.
Anthem
Aon plc
A.O. Smith Corp
APA Corporation
Apple Inc.
Applied Materials Inc.
Aptiv PLC
Archer-Daniels-Midland Co
Arista Networks
Arthur J. Gallagher & Co.
Assurant
AT&T Inc.
Atmos Energy
Autodesk Inc.
Automatic Data Processing
AutoZone Inc
AvalonBay Communities
Avery Dennison Corp
Baker Hughes Co
Ball Corp
Bank of America C

In [36]:
env_data.sort_values(by='neg_env_count', ascending=False)

,company,neg_env_count
0,3M Company,1
18,Align Technology,1
59,Ball Corp,1
22,Alphabet Inc.,1
24,Amazon.com Inc.,1
...,...,...
35,Ametek,0
37,Amphenol Corp,0
39,"ANSYS, Inc.",0
41,Aon plc,0
